In [2]:
import sys
sys.path.append(r'E:\01_PProj\ML_LIB')

import Lib_sjw.training as tr
import Lib_sjw.model_interface as mi
import Lib_sjw.model_parmas as mp
import Lib_sjw.evaluator as ev
import Lib_sjw.classification_util as cu


from sklearn.datasets import load_boston , load_iris , load_breast_cancer
from sklearn.metrics import mean_squared_error , roc_auc_score , precision_score
from sklearn.model_selection import train_test_split
import numpy as np

from collections import OrderedDict

In [ ]:
'''
1. 데이터 만들기
2. 모델 제너레이터 만들기 
3. 모델 생성 파라미터, 피팅 파라미터 만들기 
4. 성능평가 메트릭 정의하기 :  f : y , pred -> value
5. training_fixedTest 실행하기 
6. 유틸에 테스트 데이터에 대한 성능평가, 또는 저장 등을 하기 
'''
def Test_Binary_TestFold(X , y , nfold_test , nfold_val , verbose = True):
    # name list 
    name_list = ['xgb',
                 'lgb',
                 #'cat',
                 #'ngb',
                 'rfc',
                 'svm',
                 'gpc',            
                 'lda',
                 'qda',
                 'rdg',
                 'lso',
                 'ann']
    #name_list = ['ann','rdg']
    # model_dicts , param_list , 
    model_dicts = OrderedDict()    
    model_dicts['xgb']  = mi.myXGBBinary()
    model_dicts['lgb']  = mi.myLGBMBinary()
    model_dicts['cat']  = mi.myCatBoostBinary()
    model_dicts['ngb']  = mi.myNGBoostBinary()
    model_dicts['rfc']  = mi.myRandomForestBinary()
    model_dicts['svm']  = mi.mySVMBinary()
    model_dicts['gpc']  = mi.myGPBinary()
    model_dicts['lda']  = mi.myLDABinary()
    model_dicts['qda']  = mi.myQDABinary()
    model_dicts['rdg']  = mi.myRidgeBinary()
    model_dicts['lso']  = mi.myLassoBinary()
    model_dicts['ann'] = mi.myANNBinary()
    
    param_list = OrderedDict ( )
    param_list['xgb'] = mp .param_xgb ('binary' , len(np.unique(y)) , use_gpu= False )
    param_list['lgb'] = mp .param_lgbm('binary' , len(np.unique(y)) , use_gpu= False )
    param_list['cat'] = mp .param_cat ('binary' , use_gpu= True , is_unbalance= False )
    param_list['ngb'] = mp .param_ng ('binary' )
    param_list['rfc'] = mp .param_rf ('binary' )
    param_list['svm'] = mp .param_svm ('binary' )
    param_list['gpc'] = mp .param_gpc ('binary' )
    param_list['lda'] = mp .param_lda ( )
    param_list['qda'] = mp .param_qda ( )
    param_list['rdg'] = mp .param_ridge ('binary' )
    param_list['lso'] = mp .param_lasso ('binary' )
    param_list['ann'] = mp.param_ANN( layers = (58, 6) , )
    
    params_xgb = xgb_param_best
    params_lgb = lgb_param_best
    params_cat = cat_param_best
    params_ngb = ngb_param_best
    params_rfc = rf_param_best
    params_rdg = {'normalize': True , 'alpha': 0.03}
    params_lso = {'normalize': True , 'alpha': 0.03}
    params_ann = ann_param_best
           
    param_list['xgb']= {**param_list['xgb'], **params_xgb} # 뒤의 딕셔너리가 우선이 된다. 
    param_list['lgb']= {**param_list['lgb'], **params_lgb}
    param_list['cat']= {**param_list['cat'], **params_cat}
    param_list['ngb']= {**param_list['ngb'], **params_ngb}
    param_list['rfc']= {**param_list['rfc'], **params_rfc}
    param_list['rdg']= {**param_list['rdg'], **params_rdg}
    param_list['lso']= {**param_list['lso'], **params_lso}
    param_list['ann']= {**param_list['ann'], **params_ann}
    
    #fitting parmas
    fitpm_list = OrderedDict()
    for name in name_list:
            fitpm_list[name] = {}
    #fitpm_list['lgb'] = {'early_stopping_rounds' : 12 , 'verbose' : -1}
    #fit_cat = {}
    #fit_xgb = {}
    
    # metric func
    metric_func = roc_auc_score
    
    # Result
    result_list = OrderedDict()
    auc_score_list = OrderedDict()
    for name in name_list:
        if verbose : print(name)
        test_fold_index , oof, model_lists = tr.training_Testfold_noVal('binary' , model_dicts[name] , param_list[name] , fitpm_list[name] ,  metric_func , X , y , nfold_test , 517 , verbose ) 
        result_list[name] = [test_fold_index , oof, model_lists]
        auc_score_list[name] = roc_auc_score(y, oof )
    print('Test_Classification_TestFold Compelte')    
    return result_list

data = load_breast_cancer()
X = data.data
y = data.target

xtrain , xtest , ytrain , ytest = train_test_split(X , y , test_size = 0.2 )

Test_Binary(xtrain , xtest , ytrain , ytest)
Test_Binary_TestFold(X,y,5,5)